<a href="https://colab.research.google.com/github/ymirandan/projetosPython/blob/main/DS%3A%20Classificacao%20Venda%20de%20Carros%20e%20Validacao%20Cruzada.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center>
    <hr>
    <h1> 7º algoritimo: Classificação de Venda de Carros: </h1>
    <br>
    <hr>
    <h2> Objetivo: A partir de 3 informações relacionadas ao carro (milhagem por ano, ano do modelo e preço), criar um algoritimo que classifique o carro como vendido ou não </h2>
    <br>
    <I> Feito por Yasmin Miranda </I>
    <hr>
</center>

<center>
    <font size = "4" color = "#003666"> <B> Importando bibliotecas necessárias: </B> </font>
</center>

In [46]:
#!pip install graphviz
#import graphviz
import pandas as pd
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

<center>
<font size = "4" color = "#003666"> <B> Criando a base de dados: </B> </font>
</center>

In [47]:
uri = "https://gist.githubusercontent.com/guilhermesilveira/4d1d4a16ccbf6ea4e0a64a38a24ec884/raw/afd05cb0c796d18f3f5a6537053ded308ba94bf7/car-prices.csv"
dados = pd.read_csv(uri)
dados.head()
#coluna0: id do carro
#coluna2: milhagem por ano
#coluna3: ano do modelo
#coluna4: preço que o dono do carro quer vendê-lo
#coluna5: se foi vendido ou não

,Unnamed: 0,mileage_per_year,model_year,price,sold
0,0,21801,2000,30941.02,yes
1,1,7843,1998,40557.96,yes
2,2,7109,2006,89627.50,no
3,3,26823,2015,95276.14,no
4,4,7935,2014,117384.68,yes


In [48]:
#traduzir colunas para portugues
mapa = {
    "mileage_per_year":"milhagem_por_ano",
    "model_year":"ano_do_modelo",
    "price":"preco",
    "sold":"vendido"
}
dados = dados.rename(columns=mapa)

In [49]:
#trabalhar com respostas binarias:
troca = {
    "no":0,
    "yes":1
}
dados['vendido'] = dados.vendido.map(troca)
dados.head()

,Unnamed: 0,milhagem_por_ano,ano_do_modelo,preco,vendido
0,0,21801,2000,30941.02,1
1,1,7843,1998,40557.96,1
2,2,7109,2006,89627.50,0
3,3,26823,2015,95276.14,0
4,4,7935,2014,117384.68,1


Como o valor do carro está associado a sua idade, iremos adicionar essa informação na base de dados:

In [50]:
from datetime import datetime
ano_atual = datetime.today().year
dados['idade_do_modelo'] = ano_atual - dados.ano_do_modelo
dados.head()

,Unnamed: 0,milhagem_por_ano,ano_do_modelo,preco,vendido,idade_do_modelo
0,0,21801,2000,30941.02,1,24
1,1,7843,1998,40557.96,1,26
2,2,7109,2006,89627.50,0,18
3,3,26823,2015,95276.14,0,9
4,4,7935,2014,117384.68,1,10


Como no Brasil utilizamos quiilometros, vamos converter a milhagem na base de dados:

In [51]:
dados['km_por_ano'] = dados.milhagem_por_ano * 1.6094
dados.head()

,Unnamed: 0,milhagem_por_ano,ano_do_modelo,preco,vendido,idade_do_modelo,km_por_ano
0,0,21801,2000,30941.02,1,24,35086.5294
1,1,7843,1998,40557.96,1,26,12622.5242
2,2,7109,2006,89627.50,0,18,11441.2246
3,3,26823,2015,95276.14,0,9,43168.9362
4,4,7935,2014,117384.68,1,10,12770.5890


In [52]:
dados = dados.drop(columns=["Unnamed: 0","milhagem_por_ano","ano_do_modelo"],axis=1)
dados.head()

,preco,vendido,idade_do_modelo,km_por_ano
0,30941.02,1,24,35086.5294
1,40557.96,1,26,12622.5242
2,89627.50,0,18,11441.2246
3,95276.14,0,9,43168.9362
4,117384.68,1,10,12770.5890


<center>
<font size = "4" color = "#003666"> <B> Treino Algoritmo Linear: </B> </font>
</center>

In [53]:
x = dados[['preco','idade_do_modelo',"km_por_ano"]]
y = dados['vendido']

from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score

SEED = 5
np.random.seed(SEED)
treino_x, teste_x, treino_y, teste_y = train_test_split(x, y, test_size = 0.25,
                                                         stratify = y)
print("Treinaremos com %d elementos e testaremos com %d elementos" % (len(treino_x), len(teste_x)))

modelo = LinearSVC()
modelo.fit(treino_x, treino_y)
previsoes = modelo.predict(teste_x)

acuracia = accuracy_score(teste_y, previsoes) * 100
print("A acurácia foi %.2f%%" % acuracia)

Treinaremos com 7500 elementos e testaremos com 2500 elementos
A acurácia foi 46.88%


/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Iremos avaliar um algoritmo Dummy como referencia, cujas predições são 0 ou 1 respeitando apenas as proporções de 0 e 1 do treino, sem levar outras informações em consideração, para saber se nosso modelo é bom (acuracia maior que do dummy)

In [54]:
from sklearn.dummy import DummyClassifier

dummy_stratified = DummyClassifier()

dummy_stratified.fit(treino_x,treino_y)

acuracia = dummy_stratified.score(teste_x,teste_y)*100

print("A acurácia do dummy stratified foi de %.2f%%" % acuracia)

A acurácia do dummy stratified foi de 58.00%


Necessário melhorar o algoritmo, visto que a acuracia de um modelo bobo é maior que a do LinearSVC.

<center>
<font size = "4" color = "#003666"> <B> Testando Algoritmo Não Linear: </B> </font>
</center>

In [55]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

SEED = 5
np.random.seed(SEED)
raw_treino_x, raw_teste_x, treino_y, teste_y = train_test_split(x, y, test_size = 0.25,
                                                         stratify = y)
print("Treinaremos com %d elementos e testaremos com %d elementos" % (len(treino_x), len(teste_x)))

scaler = StandardScaler()
scaler.fit(raw_treino_x)
treino_x = scaler.transform(raw_treino_x)
teste_x = scaler.transform(raw_teste_x)

modelo = SVC()
modelo.fit(treino_x, treino_y)
previsoes = modelo.predict(teste_x)

acuracia = accuracy_score(teste_y, previsoes) * 100
print("A acurácia foi %.2f%%" % acuracia)

Treinaremos com 7500 elementos e testaremos com 2500 elementos
A acurácia foi 77.48%


Ganho de acurácia muito maior, comparado com o Linear e com o Dummy. Entretanto, não conseguimos dizer com muita clareza qual o peso de cada variavel na predição de venda.

<center>
<font size = "4" color = "#003666"> <B> Testando Algoritmos de Arvore de Decisao: </B> </font>
</center>

In [56]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

SEED = 5
np.random.seed(SEED)
raw_treino_x, raw_teste_x, treino_y, teste_y = train_test_split(x, y, test_size = 0.25,
                                                         stratify = y)
print("Treinaremos com %d elementos e testaremos com %d elementos" % (len(treino_x), len(teste_x)))

modelo = DecisionTreeClassifier(max_depth=2)
modelo.fit(raw_treino_x, treino_y)
previsoes = modelo.predict(raw_teste_x)

acuracia = accuracy_score(teste_y, previsoes) * 100
print("A acurácia foi %.2f%%" % acuracia)

Treinaremos com 7500 elementos e testaremos com 2500 elementos
A acurácia foi 76.84%


A acurácia foi maior que com o SVC, e é possivel identificar o que está fazendo o algoritmo tomar decisões, qual o peso de cada variavel e quais valores são mais provaveis de classificar o carro como vendido ou não. alem disso, é possivel definir a quantidade de etapas ate a decisao final

In [57]:
#gerar grafico
#from sklearn.tree import export_graphviz
#import graphviz
#features = x.columns
#dot_data = export_graphviz(modelo, out_file=None,
#                          feature_names = features,
#                           filled=True,rounded=True, class_names = ["não","sim"]) #export retorna o formato de um gráfico dotdata
#grafico = graphviz.Source(dot_data)
#grafico

Validando o Algoritmo com Validação Cruzada

In [58]:
from sklearn.model_selection import cross_validate

SEED=158020
np.random.seed(SEED)

modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo,x,y,cv=3,return_train_score=False)
results["test_score"]

array([0.75704859, 0.7629763 , 0.75337534])

In [59]:
media = results["test_score"].mean()
desvio_padrao = results["test_score"].std()
print("Accuracy com cross validation igual a 3 = [%.2f,%.2f]" % ((media - 2*desvio_padrao)*100, ((media +2*desvio_padrao)*100)))

Accuracy com cross validation igual a 3 = [74.99,76.57]


mesmo alterando o SEED, o intervalo de confiança para accuracy se mantem:

In [60]:
SEED=472393
np.random.seed(SEED)

modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo,x,y,cv=3,return_train_score=False)
results["test_score"]

media = results["test_score"].mean()
desvio_padrao = results["test_score"].std()
print("Accuracy com cross validation igual a 3 = [%.2f,%.2f]" % ((media - 2*desvio_padrao)*100, ((media +2*desvio_padrao)*100)))

Accuracy com cross validation igual a 3 = [74.99,76.57]


Alterando o Cross Validation:

In [61]:
SEED=158020
np.random.seed(SEED)

modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo,x,y,cv=5,return_train_score=False)
results["test_score"]

media = results["test_score"].mean()
desvio_padrao = results["test_score"].std()
print("Accuracy com cross validation igual a 5 = [%.2f,%.2f]" % ((media - 2*desvio_padrao)*100, ((media +2*desvio_padrao)*100)))

Accuracy com cross validation igual a 5 = [75.21,76.35]


A literatura sugere o uso do cv igual ou maior que 5 e menor ou igual a 10.

Abaixo vamos utilizar cv=10, mas nao esperamos muita diferença pois o processo de cross validation está deterministico. a aleatoriedade esta apenas no deciontreeclassifier

In [62]:
from sklearn.model_selection import KFold
SEED=301
np.random.seed(SEED)

cv=KFold(n_splits=10)
modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo,x,y,cv=cv,return_train_score=False)

In [63]:
def imprime_resultados(results):
  media = results["test_score"].mean()
  desvio_padrao = results["test_score"].std()
  print("Accuracy %.2f" % (media*100))
  print("Accuracy com cross validation = [%.2f,%.2f]" % ((media - 2*desvio_padrao)*100, ((media +2*desvio_padrao)*100)))

In [64]:
imprime_resultados(results)

Accuracy 75.78
Accuracy com cross validation = [74.37,77.19]


Aplicando aleatoriedade no cross validate:

In [65]:
from sklearn.model_selection import KFold
SEED=301
np.random.seed(SEED)

cv=KFold(n_splits=10, shuffle=True)
modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo,x,y,cv=cv,return_train_score=False)

In [66]:
imprime_resultados(results)

Accuracy 75.76
Accuracy com cross validation = [73.26,78.26]


Simulando situação horrivel de azar ou uma proporção de exemplos desbalanceados entre as classes:

In [67]:
dados_azar = dados.sort_values("vendido",ascending=True)

x = dados_azar[['preco','idade_do_modelo',"km_por_ano"]]
y = dados_azar['vendido']

dados_azar.head(2)

,preco,vendido,idade_do_modelo,km_por_ano
4999,74023.29,0,18,24813.7292
5322,84843.49,0,19,23096.4994


In [68]:
from sklearn.model_selection import KFold
SEED=301
np.random.seed(SEED)

cv=KFold(n_splits=10)
modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo,x,y,cv=cv,return_train_score=False)
imprime_resultados(results)

Accuracy 57.84
Accuracy com cross validation = [34.29,81.39]


In [69]:
from sklearn.model_selection import KFold
SEED=301
np.random.seed(SEED)

cv=KFold(n_splits=10, shuffle=True)
modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo,x,y,cv=cv,return_train_score=False)
imprime_resultados(results)

Accuracy 75.78
Accuracy com cross validation = [72.30,79.26]


o uso do shuffle garante uma boa accuracy, de maneira geral, mesmo em uma ocasião de dados desbalanceados ou com um certo viés

Abaixo vamos utilizar a estratificação dentro do Kfold, que é preferivel quando existe diferença entre as classes:

In [70]:
from sklearn.model_selection import StratifiedKFold
SEED=301
np.random.seed(SEED)

cv=StratifiedKFold(n_splits=10, shuffle=True)
modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo,x,y,cv=cv,return_train_score=False)

imprime_resultados(results)

Accuracy 75.78
Accuracy com cross validation = [73.55,78.01]


 Nessa base, não temos a informação do modelo do carro.

 Por isso, iremos criar uma nova coluna com esse dado para aplicar um algoritmo pensando em agrupar os veiculos por modelo, ou seja, treinar o modelo com veiculos do modelo x,y,z e testar com veiculos do modelo a,b,c.

In [71]:
np.random.seed(SEED)

np.random.randint(-2,3, size = 10000) #gera n numeros aleatorios entre valor minimo e valor maximo

dados['modelo_aleatorio'] = dados.idade_do_modelo + np.random.randint(-2,3, size = 10000) #gerar um 'codigo' que representa diferentes modelos de carro

dados.head()

,preco,vendido,idade_do_modelo,km_por_ano,modelo_aleatorio
0,30941.02,1,24,35086.5294,23
1,40557.96,1,26,12622.5242,25
2,89627.50,0,18,11441.2246,16
3,95276.14,0,9,43168.9362,7
4,117384.68,1,10,12770.5890,8


In [72]:
dados.modelo_aleatorio.unique()

array([23, 25, 16,  7,  8, 19, 20, 15,  9, 21, 13, 18, 11, 22, 10, 26, 24,
       12, 28, 17, 27, 14,  6,  5])

In [73]:
dados.modelo_aleatorio.value_counts() #valor (modelo) e quantidade de vezes que ele aparece

24    892
23    780
22    764
25    740
21    705
19    670
20    651
18    586
26    559
17    546
16    514
15    414
14    382
27    372
13    297
12    290
11    209
28    206
10    152
9     122
8      70
7      57
6      19
5       3
Name: modelo_aleatorio, dtype: int64

Validação Cruzada considerando modelos de carro como identificação, ou seja, ele aparece ou no treino ou no teste.

In [74]:
from sklearn.model_selection import GroupKFold
SEED=301
np.random.seed(SEED)

cv=GroupKFold(n_splits=10)
modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo,x,y,cv=cv,groups = dados.modelo_aleatorio,return_train_score=False)

imprime_resultados(results)

Accuracy 75.74
Accuracy com cross validation = [72.87,78.61]


Validação Cruzada com StandardScaler

In [75]:
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

SEED = 301
np.random.seed(SEED)
scaler = StandardScaler()
scaler.fit(treino_x)
treino_x_escalado = scaler.transform(treino_x)
teste_x_escalado = scaler.transform(teste_x)

modelo = SVC()
modelo.fit(treino_x_escalado,treino_y)
previsoes = modelo.predict(teste_x_escalado)

acuracia = accuracy_score(teste_y, previsoes)*100
print("A acuracia foi %.2f" % acuracia)

A acuracia foi 77.48


In [76]:
from sklearn.model_selection import GroupKFold
SEED=301
np.random.seed(SEED)

cv=GroupKFold(n_splits=10)
modelo = SVC()
results = cross_validate(modelo,x,y,cv=cv,groups = dados.modelo_aleatorio,return_train_score=False)

imprime_resultados(results)

Accuracy 77.22
Accuracy com cross validation = [75.02,79.42]


Porém, é necessario criar um pipeline para escalar o treino e em seguida fazer a validação cruzada:

In [78]:
from sklearn.pipeline import Pipeline

SEED = 301
np.random.seed(SEED)

scaler = StandardScaler()
modelo = SVC()

pipeline = Pipeline([('transformacao', scaler),('estimador',modelo)])
cv=GroupKFold(n_splits=10)
modelo = SVC()
results = cross_validate(modelo,x,y,cv=cv,groups = dados.modelo_aleatorio,return_train_score=False)

imprime_resultados(results)

Accuracy 77.22
Accuracy com cross validation = [75.02,79.42]
